In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from video_processor import VideoProcessor
import datetime
import os


In [2]:
# import os
# import pandas as pd

# DATASET_DIR = '../datasets/IXMAS/'

# # Function to get list of video clips and their labels
# def get_video_clips_and_labels(dataset_dir):
#     data = {'clip_path': [], 'label': []}
    
#     for root, dirs, files in os.walk(dataset_dir):
#         # for dir_name in dirs:
#         #     label = dir_name
#         #     clips_dir = os.path.join(root, dir_name)
            
#         for clip_file in files:
#             clip_path = os.path.join(root, clip_file)
#             label = str(clip_file).split('_')[2]
#             data['clip_path'].append(clip_path)
#             data['label'].append(label)

#     return data

# # Create DataFrame and save to CSV
# data = get_video_clips_and_labels(DATASET_DIR)
# df = pd.DataFrame(data)
# csv_path = os.path.join(DATASET_DIR, 'dataset.csv')
# df.to_csv(csv_path, index=False)

# print(f"CSV file saved at: {csv_path}")


In [3]:
df = pd.read_csv('../datasets/IXMAS/dataset.csv')
df.head()

,clip_path,label
0,../datasets/IXMAS/alba1_01_check-watch_cam0_fr...,check-watch
1,../datasets/IXMAS/alba1_01_check-watch_cam1_fr...,check-watch
2,../datasets/IXMAS/alba1_01_check-watch_cam2_fr...,check-watch
3,../datasets/IXMAS/alba1_01_check-watch_cam3_fr...,check-watch
4,../datasets/IXMAS/alba1_01_check-watch_cam4_fr...,check-watch


In [4]:
df['label'].unique()

array(['check-watch', 'cross-arms', 'scratch-head', 'sit-down', 'get-up', 'turn-around', 'walk', 'wave', 'punch', 'kick', 'point', 'pick-up'], dtype=object)

# Inference on Dataset

In [5]:
from ultralytics import YOLO
import cv2

In [6]:
model = YOLO('models/yolov8s-pose.pt') 

In [8]:
batch_size = 10
vp = VideoProcessor(model,batch_size, max_frames=10, img_sz=320)

keypoints_df = pd.DataFrame()
class_keypoints_df = pd.DataFrame()
for train_df in [df]:  
    batch, skipped_id = vp.process_video_batch(train_df['clip_path'])
    batch = np.array(batch)[:,0,:].reshape(-1,batch_size , 34)
    class_keypoints_df = pd.DataFrame({ 'keypoints': batch.tolist(), 'label': train_df['label'].drop(skipped_id)})
    keypoints_df = pd.concat([keypoints_df, class_keypoints_df])
    print(f'Skipped: {len(skipped_id)}')

# keypoints_df.columns = ['keypoints', 'label']

cv2.destroyAllWindows()

Processing Videos: 100%|██████████| 1800/1800 [11:00<00:00,  2.73it/s]


Skipped: 0


In [9]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'keypoint_dataset/IXMAS/keypoints_df_{timestamp}.csv'

# Save the keypoints_df to the new file
keypoints_df.to_csv(filename, index=False)